In [1]:
import torch
from torch.utils.data import DataLoader
import pandas as pd
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset
import torch.nn as nn  
import torch.optim as optim


In [2]:
class CustomDataset(Dataset):
    def __init__(self, csv_file):
        self.data = pd.read_csv(csv_file)
        self.features = self.data.iloc[:, :-1].values
        self.targets = self.data.iloc[:, -1].values

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features[idx], dtype=torch.float32)
        target = torch.tensor(self.targets[idx], dtype=torch.float32)  
        return features, target

csv_file = "./MEFAR_MID.csv"
dataset = CustomDataset(csv_file)

train_set, test_set = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_set, batch_size=10, shuffle=True)
test_loader = DataLoader(test_set, batch_size=1, shuffle=True)


In [3]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.in_dim = 17
        self.out_dim = 1
        self.fc1 = nn.Linear(self.in_dim, 16)
        self.fc2 = nn.Linear(16, 8)
        self.fc3 = nn.Linear(8, 4)
        self.fc4 = nn.Linear(4, 2)
        self.fc5 = nn.Linear(2, self.out_dim)
        self.relu = nn.ReLU()
        self.log_softmax = nn.LogSoftmax(dim=1)
    
    def forward(self, x):
        a1 = self.relu(self.fc1(x))
        a2 = self.relu(self.fc2(a1))
        a3 = self.relu(self.fc3(a2))
        a4 = self.relu(self.fc4(a3))
        output = self.fc5(a4)  # Pas besoin de log_softmax pour la régression
        return output

In [5]:
model = MLP()
criterion = nn.MSELoss()  # Utilisation de MSELoss pour la régression
optimizer = optim.SGD(model.parameters(), lr=0.1)

for epoch in range(2):
    running_loss = 0.0
    ancienRunning=-1
    for i, data in enumerate(train_loader, 0):
        if (abs(ancienRunning-running_loss)<0.001):
            optimizer = optim.SGD(model.parameters(), lr=0.00001)
        inputs, labels = data
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs.squeeze(), labels)  # Squeeze pour éliminer les dimensions inutiles
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        
        
        if (i + 1) % 2000 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


[1,  2000] loss: 0.235
[1,  4000] loss: 0.217
[1,  6000] loss: 0.208
[1,  8000] loss: 0.184
[1, 10000] loss: 0.166
[1, 12000] loss: 0.158
[1, 14000] loss: 0.154
[1, 16000] loss: 0.143
[1, 18000] loss: 0.136
[1, 20000] loss: 0.133
[1, 22000] loss: 0.129
[1, 24000] loss: 0.124
[1, 26000] loss: 0.120
[1, 28000] loss: 0.118
[1, 30000] loss: 0.117
[1, 32000] loss: 0.114
[1, 34000] loss: 0.112
[1, 36000] loss: 0.110
[1, 38000] loss: 0.110
[1, 40000] loss: 0.109
[1, 42000] loss: 0.108
[1, 44000] loss: 0.106
[1, 46000] loss: 0.105
[1, 48000] loss: 0.103
[1, 50000] loss: 0.105
[1, 52000] loss: 0.107
[1, 54000] loss: 0.103
[1, 56000] loss: 0.104
[1, 58000] loss: 0.102
[1, 60000] loss: 0.102
[1, 62000] loss: 0.102
[1, 64000] loss: 0.101
[1, 66000] loss: 0.103
[1, 68000] loss: 0.101
[1, 70000] loss: 0.100
[1, 72000] loss: 0.098
[2,  2000] loss: 0.098
[2,  4000] loss: 0.095
[2,  6000] loss: 0.098
[2,  8000] loss: 0.096
[2, 10000] loss: 0.096
[2, 12000] loss: 0.095
[2, 14000] loss: 0.097
[2, 16000] 

In [6]:
def test_model(model, test_loader, criterion):
    model.eval()  # Mettre le modèle en mode évaluation
    test_loss = 0.0
    correct = 0
    total = 0
    with torch.no_grad():  # Désactiver le calcul des gradients pendant l'évaluation
        for inputs, labels in test_loader:
            outputs = model(inputs)
            loss = criterion(outputs.squeeze(), labels.float())  # Convertir les étiquettes en type float
            test_loss += loss.item()
            # Calcul de la précision
            predicted = torch.round(outputs).squeeze()  # Arrondir les prédictions
            #print(outputs ,"     rechercher : " , labels) 
            
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    
    # Afficher les résultats
    avg_loss = test_loss / len(test_loader)
    accuracy = correct / total
    print('Test Loss: {:.4f}, Accuracy: {:.2f}%'.format(avg_loss, accuracy * 100))

# Utiliser la fonction test_model pour évaluer le modèle
test_model(model, test_loader, criterion)


/home/ec2-user/anaconda3/envs/pytorch_p310/lib/python3.10/site-packages/torch/nn/modules/loss.py:535: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Test Loss: 0.0824, Accuracy: 89.14%


In [7]:
torch.save(model.state_dict(), 'FligneBS10.pth')